# GraphFrames

Dans ce TP, nous allons utiliser la librairie GraphFrame. Afin de pouvoir utiliser la dernière version de celle-ci, rattacher ce notebook à un cluster 15.3 ML, qui contient les librairies compatibles.

In [0]:
%sh /databricks/python3/bin/pip install graphframes

  Obtaining dependency information for graphframes from https://files.pythonhosted.org/packages/0b/27/c7c7e1ced2fe9a905f865dd91faaec2ac8a8e313f511678c8ec92a41a153/graphframes-0.6-py2.py3-none-any.whl.metadata
  Obtaining dependency information for nose from https://files.pythonhosted.org/packages/15/d8/dd071918c040f50fa1cf80da16423af51ff8ce4a0f2399b7bf8de45ac3d9/nose-1.3.7-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [0]:
from functools import reduce
from pyspark.sql import functions as F
from graphframes import GraphFrame

Il est possible de créer simplement des GraphFrames à partir de DataFrames de sommets et d'arêtes.

<br>- Le DataFrame de sommets doit contenir une colonne spéciale, nommée "id", qui spécifie des identifiants uniques pour chaque sommet dans le graphe.
<br>- Le DataFrame d'arêtes doit contenir deux colonnes spéciales : "src" (identifiant du sommet source de l'arête) et "dst" (identifiant du sommet destination de l'arête).
<br>- Les deux DataFrames peuvent avoir d'autres colonnes arbitraires. Ces colonnes peuvent représenter des attributs de sommets et d'arêtes.

On se donne la liste des personnes suivantes, caractérisée par 3 champs : l'id de la personne (en réalité du futur noeud), son nom et son âge. Ces personnes sont destinées à être les noeuds de notre futur graphe.

In [0]:
people = [
    ("a", "Alice", 34),
    ("b", "Bob", 36),
    ("c", "Charlie", 30),
    ("d", "David", 29),
    ("e", "Esther", 32),
    ("f", "Fanny", 36),
    ("g", "Gabby", 60),
]

Créer un dataframe <i>vertices</i> à partir de cette liste.

In [0]:
vertices = spark.createDataFrame(
    people,
    schema=["id", "name", "age"],
)

In [0]:
vertices.printSchema()
display(vertices)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



id,name,age
a,Alice,34
b,Bob,36
c,Charlie,30
d,David,29
e,Esther,32
f,Fanny,36
g,Gabby,60


Ces personnes sont inscrites à un réseau social où deux options sont possibles :
- suivre une personne ("follow"),
- déclarer être ami avec cette personne ("friend"), ce qui n'est pas forcément réciproque.
Il n'est pas possible de cumuler les deux status.

Voici l'ensemble des relations :
<br>- Alice a déclaré être amie avec Bob.
<br>- Bob suit Charlie.
<br>- Charlie suit Bob.
<br>- Fanny suit Charlie.
<br>- Esther suit Fanny.
<br>- Esther a déclaré être amie avec David.
<br>- David suit Esther.
<br>- David a déclaré être ami avec Alice.
<br>- Alice a déclaré être amie avec Esther.

Créer un dataframe contenant l'ensemble de ces interactions. Une interaction sera caractérisée par l'identifiant de la personne à l'origine de l'interaction ("src"), celui de la personne visée par l'interaction ("dst"), et le statut de la relation ("relationship", valant "follow" ou "friend").

In [0]:
relations = [
    ("a", "b", "friend"),
    ("b", "c", "follow"),
    ("c", "b", "follow"),
    ("f", "c", "follow"),
    ("e", "f", "follow"),
    ("e", "d", "friend"),
    ("d", "e", "follow"),
    ("d", "a", "friend"),
    ("a", "e", "friend"),
]

edges = spark.createDataFrame(
    relations,
    schema=["src", "dst", "relationship"],
)

Créer un graphe représentatif de la situation à partir de ces deux dataframes.

In [0]:
g = GraphFrame(vertices, edges)
print(g)

GraphFrame(v:[id: string, name: string ... 1 more field], e:[src: string, dst: string ... 1 more field])



## Requêtes de base sur les graphes et les DataFrames
Les objets de type GraphFrame fournissent plusieurs méthodes natives pour manipuler les graphes. Nous allons les manipuler ici.

Afficher les sommets/noeuds du graphe, ainsi que ses arcs. Afin de savoir dans quels attributs de notre objet ils sont stockés, consulter la documentation de la classe GraphFrame : https://graphframes.github.io/graphframes/docs/_site/api/python/graphframes.html.

In [0]:
display(g.vertices)
display(g.edges)

id,name,age
a,Alice,34
b,Bob,36
c,Charlie,30
d,David,29
e,Esther,32
f,Fanny,36
g,Gabby,60


src,dst,relationship
a,b,friend
b,c,follow
c,b,follow
f,c,follow
e,f,follow
e,d,friend
d,e,follow
d,a,friend
a,e,friend


Déterminer le "degré entrant" de l'ensemble des sommets (i.e. pour chaque sommet, le nombre de relations qui pointent vers ce sommet).

In [0]:
display(g.inDegrees)

id,inDegree
b,2
c,2
f,1
d,1
e,2
a,1


Déterminer le degré sortant de l'ensemble des sommets.

In [0]:
display(g.outDegrees)

id,outDegree
a,2
b,1
c,1
f,1
e,2
d,2


Déterminer le degré des sommets (somme des degrés entrant et sortant).

In [0]:
display(g.degrees)

id,degree
b,3
a,3
c,3
f,2
e,4
d,3


Il est possible d'exécuter directement des requêtes sur le DataFrame des sommets.
Trouver l'âge de la personne la plus jeune dans le graphe. Utiliser le DSL.

In [0]:
youngest = g.vertices.groupBy().min("age")
youngest.show()

+--------+
|min(age)|
+--------+
|      29|
+--------+



Il est bien entendu également possible d'exécuter des requêtes sur le DataFrame des arcs.
Compter le nombre de relations de type "follow" dans le graphe.

In [0]:
num_follows = g.edges.filter(g.edges.relationship == "follow").count()
print(num_follows)

5


## Trouver des motifs

En utilisant des motifs, il est possible construire des relations plus complexes impliquant des arêtes et des sommets.

<br>Les motifs à rechercher sont dénotés par des expressions.
<br>Un expression élémentaire est "(a)-[e]->(b)". Elle signifie qu'il existe une arête dirigée de a vers b.
<br>Il est possible de combiner ces expressions (le symbole ; est utilisé pour exprimer le ET logique).
<br>Pour les sommets, la répétition d'une lettre signifie que la référence se fait à un même sommet.
<br> Pour les arêtes, il n'est pas possible de répéter la même lettre.
<br>Une fois l'expression construite, il faut appeler la méthode find de la manière suivante : graph.find(motif)

Trouver toutes les paires de sommets avec des arêtes dans les deux directions entre eux (les relations directe et réciproque ne sont pas forcément les mêmes). Le résultat doit être un DataFrame, dans lequel les noms de colonnes sont donnés par les clés du motif.

In [0]:
expr = "(a)-[e1]->(b); (b)-[e2]->(a)"
motifs = g.find(expr)
display(motifs)

a,e1,b,e2
"List(c, Charlie, 30)","List(c, b, follow)","List(b, Bob, 36)","List(b, c, follow)"
"List(b, Bob, 36)","List(b, c, follow)","List(c, Charlie, 30)","List(c, b, follow)"
"List(d, David, 29)","List(d, e, follow)","List(e, Esther, 32)","List(e, d, friend)"
"List(e, Esther, 32)","List(e, d, friend)","List(d, David, 29)","List(d, e, follow)"


Puisque le résultat est un DataFrame, il est possible d'exécuter des requêtes par dessus le motif.
Parmi les relations précédentes, déterminer celles qui concernent deux personnes dont l'une au moins est âgée de 34 ans ou plus.

In [0]:
filtered_motifs = motifs.filter("a.age >= 34 or b.age >= 34")
display(filtered_motifs)

a,e1,b,e2
"List(c, Charlie, 30)","List(c, b, follow)","List(b, Bob, 36)","List(b, c, follow)"
"List(b, Bob, 36)","List(b, c, follow)","List(c, Charlie, 30)","List(c, b, follow)"



## Requêtes stateful
La plupart des requêtes motif sont sans état et simples à exprimer, comme dans notre exemple précédent. Parfois, une requête plus complexe doit transporter un état le long d'un chemin dans le motif. On peut l'exprimer en combinant la recherche de motifs GraphFrame avec des filtres sur le résultat utilisant des opérations de séquence, agissant sur les colonnes du DataFrame.

Exemple : nous souhaitions identifier les chaînes de 4 sommets a->b->c->d (donc 3 arêtes) vérifiant une certaine propriété définie par une séquence de fonctions. Le processus sera le suivant :
<br> 1. Initialiser l'état sur le chemin.
<br> 2. Mettre à jour l'état en fonction du sommet a.
<br> 3. Mettre à jour l'état en fonction du sommet b.
<br> 4. Mettre à jour l'état en fonction du sommet c.
<br> 5. Et enfin, mettre à jour l'état en fonction du sommet d.

Si l'état final correspond à nos conditions, alors le filtre accepte la chaîne.

Identifier les chaînes de 4 sommets où au moins 2 des 3 arêtes sont des relations "friend". On suivra l'état suivant : nombre actuel d'arêtes "friend".
Ne pas oublier qu'il est possible d'utiliser les fonctions de F (functions) importé en début de notebook.

In [0]:
chain_4 = g.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[cd]->(d)")

def cum_friends(cnt, edge):
  relationship = F.col(edge)["relationship"]
  return F.when(relationship == "friend", cnt + 1).otherwise(cnt)

edges = ["ab", "bc", "cd"]
num_friends = reduce(cum_friends, edges, F.lit(0))

chain_2_friends = chain_4.withColumn("num_friends", num_friends).filter(num_friends >= 2)
display(chain_2_friends)

a,ab,b,bc,c,cd,d,num_friends
"List(d, David, 29)","List(d, e, follow)","List(e, Esther, 32)","List(e, d, friend)","List(d, David, 29)","List(d, a, friend)","List(a, Alice, 34)",2
"List(a, Alice, 34)","List(a, e, friend)","List(e, Esther, 32)","List(e, d, friend)","List(d, David, 29)","List(d, a, friend)","List(a, Alice, 34)",3
"List(e, Esther, 32)","List(e, d, friend)","List(d, David, 29)","List(d, a, friend)","List(a, Alice, 34)","List(a, b, friend)","List(b, Bob, 36)",3
"List(d, David, 29)","List(d, a, friend)","List(a, Alice, 34)","List(a, b, friend)","List(b, Bob, 36)","List(b, c, follow)","List(c, Charlie, 30)",2
"List(d, David, 29)","List(d, a, friend)","List(a, Alice, 34)","List(a, e, friend)","List(e, Esther, 32)","List(e, d, friend)","List(d, David, 29)",3
"List(e, Esther, 32)","List(e, d, friend)","List(d, David, 29)","List(d, e, follow)","List(e, Esther, 32)","List(e, d, friend)","List(d, David, 29)",2
"List(e, Esther, 32)","List(e, d, friend)","List(d, David, 29)","List(d, a, friend)","List(a, Alice, 34)","List(a, e, friend)","List(e, Esther, 32)",3
"List(a, Alice, 34)","List(a, e, friend)","List(e, Esther, 32)","List(e, d, friend)","List(d, David, 29)","List(d, e, follow)","List(e, Esther, 32)",2
"List(d, David, 29)","List(d, a, friend)","List(a, Alice, 34)","List(a, e, friend)","List(e, Esther, 32)","List(e, f, follow)","List(f, Fanny, 36)",2



## Sous-graphes
GraphFrames fournit une API pour construire des sous-graphes en filtrant sur les arêtes et les sommets.
<br>A partir de notre graphe complet, construire le graphe n'incluant que les personnes de strictement plus de 30 ans et qui ont des amis de strictement plus de 30 ans. Ne garder dans le graphe que les relations de type "friend".
<br>S'aider de la documentation pour filtrer les sommets et les arêtes. Indication : il existe une méthode pour supprimer les Objets isolés une fois les sommets et arêtes filtrés.

In [0]:
g2 = g.filterEdges("relationship = 'friend'").filterVertices("age  > 30").dropIsolatedVertices()

In [0]:
display(g2.vertices)
display(g2.edges)

id,name,age
a,Alice,34
b,Bob,36
e,Esther,32


src,dst,relationship
a,b,friend
a,e,friend



## Algorithmes de graphes classiques

GraphFrames fournit un certain nombre d'algorithmes "built-in", dont les plus notables sont :
* Breadth-first search (BFS)
* Connected components
* Strongly connected components
* Label Propagation Algorithm (LPA)
* PageRank (classique et personnalisé)
* Shortest paths
* Triangle count

Dans cette formation, nous nous intéresserons à PageRank et Shortest paths.

## PageRank

Le but de PageRank est d'identifier les sommets "importants".

De votre point de vue, quel sommet du graphe est le plus important ?
<br>Lancer l'algorithme PageRank sur notre graphe avec les paramètres suivants :
<br>resetProbability=0.15
<br>tol=0.01
<br>Afficher les résultats.

In [0]:
# Votre code ici

Contrairement au PageRank standard, qui calcule les scores de pertinence en fonction de la structure globale du graphe, le PageRank personnalisé prend en compte les préférences ou les priorités spécifiques de l'utilisateur.
<br> Dans ce contexte de PageRank personnalisé, sourceId permet de régler le noeud (via son identifiant) à partir duquel l'algorithme commence à évaluer la pertinence des autres nœuds du graphe.
<br> Relancer l'algorithme PageRank en choisissant le noeud "f" correspondant à Fanny comme sourceId.

In [0]:
# Votre code ici

Essayer d'expliquer le résultat.
A votre avis, quelle(s) propriété(s) de graphe(s) rendent le point de départ d'autant plus important ?


## Shortest paths

Calcule les chemins les plus courts vers un ensemble donné de sommets "repères" (landmarks en anglais).

A l'aide de l'algorithme shortestPaths, calculer , pour chaque sommet, la longueur du plus court chemin reliant ce sommet au sommet "a" et celle du plus court chemin reliant ce sommet au sommet "d" (s'il existe de tels chemins).

In [0]:
results = g.shortestPaths(landmarks=["a", "d"])
display(results)

id,name,age,distances
g,Gabby,60,Map()
f,Fanny,36,Map()
e,Esther,32,"Map(a -> 2, d -> 1)"
d,David,29,"Map(a -> 1, d -> 0)"
c,Charlie,30,Map()
b,Bob,36,Map()
a,Alice,34,"Map(a -> 0, d -> 2)"


Bonus : Choisir un algorithme supplémentaire dans la liste des algorithmes classiques et le tester !

In [0]:
connected_components = g.stronglyConnectedComponents(maxIter=5)
connected_components.show()
 

+---+-------+---+-------------+
| id|   name|age|    component|
+---+-------+---+-------------+
|  g|  Gabby| 60| 146028888064|
|  f|  Fanny| 36| 412316860416|
|  e| Esther| 32| 670014898176|
|  d|  David| 29| 670014898176|
|  c|Charlie| 30|1047972020224|
|  b|    Bob| 36|1047972020224|
|  a|  Alice| 34| 670014898176|
+---+-------+---+-------------+



In [0]:
connected_components = g.connectedComponents()
connected_components.show()
 

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1427304125574295>, line 1
----> 1 connected_components = g.connectedComponents()
      2 connected_components.show()

File /databricks/jars/spark--maven-trees--ml--15.x--graphframes--org.graphframes--graphframes_2.12--org.graphframes__graphframes_2.12__0.8.3-db1-spark3.5.jar/graphframes/graphframe.py:345, in GraphFrame.connectedComponents(self, algorithm, checkpointInterval, broadcastThreshold, optStartIter, intermediateStorageLevel, sparsityThreshold)
    316 """
    317 Computes the connected components of the graph.
    318 
   (...)
    335 :return: DataFrame with new vertices column "component"
    336 """
    337 javaIntermediateStorageLevel = self._sc._getJavaStorageLevel(intermediateStorageLevel)
    338 jdf = self._jvm_graph.connectedComponents() \
    339     .setAlgorithm(algorithm) \
    340     .setCheckpointI

In [0]:
!pwd

/databricks/driver


In [0]:
!mkdir /databricks/driver/checkpoints

In [0]:
spark.sparkContext.setCheckpointDir('/databricks/driver/checkpoints')

In [0]:
connected_components = g.connectedComponents()
connected_components.show()


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:720)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:439)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:439)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio